In [ ]:
from classes import airspace

as_switzerland = airspace(id = 'rectangle_Switzerland',
                          volume=(45.52084813133491, 
                                  47.85, 
                                  5.85392, 
                                  10.80539976710652, 
                                  18000, 
                                  60000,),
                                  )

In [ ]:
as_switzerland.generate_cells()
as_switzerland.visualise_cells()

In [ ]:
as_switzerland

In [ ]:
as_switzerland.cubes[0].visualise()

In [ ]:
x = "2019-01-12 hh:mm:ss"

In [ ]:
x[5:7]

In [ ]:
as_switzerland.generate_cells()
as_switzerland.visualise_cells()

In [ ]:
as_switzerland.cubes[0].visualise()

In [ ]:
from traffic.core import Traffic
trajs = Traffic.from_file(f"data/rectangle_Switzerland/02_combined/combined_2019_12.parquet")

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

trajs_resamp =(
    trajs.clean_invalid()
    .assign_id()
    .filter()
    .resample("5s")
    .eval(max_workers= 20, desc='resampling')
)

In [ ]:
trajs.data.head().dtypes

In [ ]:
from traffic.core import Traffic
x = trajs.data.dtypes.to_dict()
del x['last_position']
new = trajs_resamp.data.astype(x)
trajs_prep = Traffic(new)

In [ ]:
trajs_prep = Traffic(new)
trajs_prep.data.head().dtypes

In [ ]:
import os

save_path = f"data/rectangle_Switzerland/03_preprocessed"
if os.path.isdir(save_path) is False:
    os.mkdir(save_path)
trajs_prep.to_parquet(f"{save_path}/preprocessed.parquet")

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from cartes.crs import CH1903p

t_gmm = t_unwrapped.clustering(
    nb_samples=15,
    projection=CH1903p(),
    features=["x", "y", "track_unwrapped"],
    clustering=GaussianMixture(n_components=30),
    transform=StandardScaler(),
).fit_predict()

In [ ]:
dict(t_gmm.groupby(["cluster"]).agg({"flight_id": "nunique"}).flight_id)

In [ ]:
n_clusters = 1 + t_gmm.data.cluster.max()

In [ ]:
from ipyleaflet import Map, basemaps, MeasureControl, Polyline
from ipywidgets import Layout
from traffic.data import nm_airspaces

map_ = Map(
    center=(47.48891, 8.85743),
    zoom=8,
    basemap=basemaps.CartoDB.Positron,
    layout=Layout(width="1000px", height="1000px"),
)

# get list of length x with unique colors
color_cycle = fn.generate_color_list(n_clusters)

for cluster in range(n_clusters):
    for flight in t_gmm.query(f"cluster == {cluster}"):
        map_.add_layer(flight, weight=1, color = color_cycle[cluster], opacity=0.3)
map_.add_layer(nm_airspaces["LSAZM7"], weight=2, color='black', fill = False)
map_

In [ ]:
from traffic.core.projection import EuroPP

t = (
    t_gmm.query(f"cluster == {5}")
    .assign_id()
    .unwrap()
    .resample(100)
    .eval()
)

t = t.compute_xy(projection=EuroPP())

In [ ]:
from traffic.core import Traffic

def compute_timedelta(df: "pd.DataFrame"):
    return (df.timestamp - df.timestamp.min()).dt.total_seconds()

t = t.iterate_lazy().assign(timedelta=compute_timedelta).eval()

In [ ]:
df = t.data
df_new = df[np.isfinite(df[["x", "y", "altitude", "timedelta"]]).all(1)]
df_new = df_new.drop_duplicates(subset="timestamp", keep="first")
t = Traffic(df_new)

t = (
    t
    .assign_id()
    .unwrap()
    .resample(100)
    .eval()
)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from traffic.algorithms.generation import Generation

g1 = Generation(
    generation=GaussianMixture(n_components=4),
    features=["x", "y", "altitude", "timedelta"],
    scaler=MinMaxScaler(feature_range=(-1, 1))
).fit(t)

In [ ]:
t_gen1 = g1.sample(200, projection=EuroPP())

In [ ]:
# t = (
#     t
#     .drop_duplicates(subset="timestamp", keep="first")
#     .resample(200)
#     .eval(max_workers=2)
# )
# t_gen1 = (
#     t_gen1
#     .drop_duplicates(subset="timestamp", keep="first")
#     .resample(200)
#     .eval(max_workers=2)
# )
# t_gen1 = t_gen1.clip(nm_airspaces["LSAZM7"].shape.bounds).eval(desc="clipping", max_workers=2)
# t = t.clip(nm_airspaces["LSAZM7"].shape.bounds).eval(desc="clipping", max_workers=2)

In [ ]:
from ipyleaflet import Map, basemaps, MeasureControl, Polyline
from ipywidgets import Layout
from traffic.data import nm_airspaces

map_ = Map(
    center=(47.48891, 8.85743),
    zoom=8,
    basemap=basemaps.CartoDB.Positron,
    layout=Layout(width="1500px", height="1500px"),
)

# get list of length x with unique colors
color_cycle = fn.generate_color_list(n_clusters)

for flight in t_gen1:
    map_.add_layer(flight, weight=1, color = 'red', opacity=0.3)

for flight in t:
    map_.add_layer(flight, weight=1, color = 'blue', opacity=0.3)

map_.add_layer(nm_airspaces["LSAZM7"], weight=2, color='black', fill = False)
map_

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter())

for flight in t:
    fig.add_trace(go.Scatter(
        mode="lines",
        x=[*range(len(flight.data))],
        y=flight.data.altitude,
        line=dict(color='rgba(0, 0, 255, 0.3)', width=1)
    ))

for flight in t_gen1:
    fig.add_trace(go.Scatter(
        mode="lines",
        x=[*range(len(flight.data))],
        y=flight.data.altitude,
        line=dict(color='rgba(255, 0, 0, 0.3)', width=1)
    ))

fig.show()

# 217 # provided input
# 218 f i g = go . Figure (
# 219 data=go . S c a t t e r ( x=given . time , y=given . a l t , name=" input " , mode="markers+ l i n e s " )
# 220 )
# 221 # predi c ted futur e a l t i t ud e
# 222 f i g . add_t race (
# 223 go . S c a t t e r (
# 224 x=predi c ted . time , y=predi c ted . a l t , name=" predi c ted " , mode="markers+ l i n e s "
# 225 )
# 226 )
# 227 #

In [ ]:

# Reduction to low traffic trajectories-------------------------------------------------
def get_lowtraf_trajs(file_load: str,
                      path_save: str,
                      max_percentile: float = 0.99,
                      low_th: float = 0.2,
                      max_workers: int = 20,
                      resampling: str = '5s') -> None:
    """
    Computes hours of low traffic and reduces the trajectories to these hours. The hours
    of low traffic are defined as the hours with a traffic volume below a threshold
    which is defined as a fraction of the maximum traffic volume. The maximum traffic
    volume is defined as a percentile of flown seconds per hour.

    Parameters
    ----------
    file_load : str
        Path to the traffic file which will be loaded
    path_save : str
        Path to the folder where the low traffic trajectories will be saved
    max_percentile : float, optional
        Percentile which is defined as max traffic, by default 0.99
    low_th : float, optional
        Threshold expressed as a fraction of max traffic below which an hour will be
        labeled as low traffic-hour, by default 0.2
    max_workers : int, optional
        Max amount of workers for multi-processing of resampling and id assignment, 
        by default 20
    resampling : str, optional
        Resampling interval applied to the trajectories, by default '5s'
    """
    # Load data
    trajs = Traffic.from_file(file_load)
    # Id assignment and resampling
    trajs = trajs.assign_id().eval()
    # Aggregation on trajectory level, computation of stay time and hour entered
    df = trajs.resample(resampling).eval(desc='processing', max_workers=max_workers).data
    df = df.groupby('flight_id')['timestamp'].agg(['min', 'max']).reset_index()
    df = df.rename({'min': 'in', 'max': 'out'}, axis=1)
    df['stay_s'] = (df['out'] - df['in']).dt.total_seconds()
    df['timestamp_entered_h'] = df['in'].dt.floor('h')
    df = df.drop(['in','out'], axis=1)
    # Aggreagation on hourly level
    hourly_time = df.groupby(['timestamp_entered_h'])['stay_s'].sum()
    hourly_count = df.groupby(['timestamp_entered_h'])['flight_id'].count()
    hourly_df = pd.concat([hourly_time, hourly_count], axis=1)
    hourly_df = hourly_df.rename({'flight_id': 'count'}, axis=1)
    # Rescaling and identification of hours below threshold
    hourly_df= hourly_df/hourly_df.quantile(max_percentile)
    hourly_df['low'] = hourly_df['stay_s'].apply(lambda x: 'yes' if x < low_th and x >= 0 else 'no')
    low_hours = hourly_df[hourly_df.low == 'yes'].index
    # Reduction of trajectories to low traffic hours
    ids_use = df[df.timestamp_entered_h.isin(low_hours)].flight_id.to_numpy()
    trajs_use = trajs[ids_use]
    # create path_save if it does not exist
    if os.path.isdir(path_save) == False:
        os.mkdir(path_save)
    # save the Traffic object as a parquet file
    trajs_use.to_parquet(f'{path_save}/low_traffic.parquet')

# Restructuring to training data--------------------------------------------------------------------

def get_training_data(file_load: str,
                        path_save: str,
                        max_workers: int = 20) -> None:
    """
    Restructures the trajectories to training data. The trajectories are resampled to 100 data
    points per trajectory. The data points are interpolated linearly and the resulting trajectories
    are converted to numpy arrays min-max rescaling is applied. The resulting numpy arrays
    (normalized and non-normalized) are saved along with a txt file containing the min and max
    values used for the normalization.

    Parameters
    ----------
    file_load : str
        Path to the traffic file which will be loaded
    path_save : str
        Path to the folder where the created files will be saved
    max_workers : int, optional
        Max amount of workers for multi-processing of resampling, by default 20
    """
    
    # Load data
    trajs = Traffic.from_file(file_load)

    # Resample to 100 data points per trajectory
    trajs = trajs.resample(100).eval(max_workers=max_workers)

    # Convert to numpy array
    X = []
    for flight in tqdm(trajs):
        df = flight.data[['latitude', 'longitude', 'altitude']]
        df = df.interpolate(method='linear', limit_direction='both').ffill().bfill()
        df_as_np = df.to_numpy()
        X.append(df_as_np)
    X = np.array(X)

    # Remove potential NaNs
    indexList_X_nan = [np.any(i) for i in np.isnan(X)]
    X = np.delete(X, indexList_X_nan, axis=0)
    X.shape

    # Min-Max Normalization
    lat_max = np.max(X[:,:,0])
    lat_min = np.min(X[:,:,0])
    lon_max = np.max(X[:,:,1])
    lon_min = np.min(X[:,:,1])
    alt_max = np.max(X[:,:,2])
    alt_min = np.min(X[:,:,2])
    X_norm = X.copy() 
    X_norm[:,:,0] = (X_norm[:,:,0] - lat_min) / (lat_max - lat_min)
    X_norm[:,:,1] = (X_norm[:,:,1] - lon_min) / (lon_max - lon_min)
    X_norm[:,:,2] = (X_norm[:,:,2] - alt_min) / (alt_max - alt_min)

    if not os.path.exists(path_save):
        os.makedirs(path_save)
    np.save(f'{path_save}/X', X)
    np.save(f'{path_save}/X_norm', X_norm)
    with open(f"{path_save}/normalisation.txt", "w") as f:
        f.write(
            'lat_min'
            + " "
            + 'lat_max'
            + " "
            + 'lon_min'
            + " "
            + 'lon_max'
            + " "
            + 'alt_min'
            + " "
            + 'alt_max'
            + f'\n{lat_min}'
            + " "
            + str(lat_max)
            + " "
            + str(lon_min)
            + " "
            + str(lon_max)
            + " "
            + str(alt_min)
            + " "
            + str(alt_max)
        )



# Visualisation-------------------------------------------------------------------------------------
def generate_color_list(num_colors):
    # Generate a list of evenly spaced hues
    hues = [i / float(num_colors) for i in range(num_colors)]

    # Shuffle the list of hues
    random.shuffle(hues)

    # Convert the hues to RGB colors
    colors = [tuple(int(i * 255) for i in colorsys.hsv_to_rgb(hue, 0.8, 0.8)) for hue in hues]

    # Convert the RGB colors to hex strings
    hex_colors = [f"#{r:02x}{g:02x}{b:02x}" for r, g, b in colors]

    return hex_colors